# formata-texto
Esse script recebe arquivos .txt extraídos do site da Câmara dos Deputados e executa operações de limpeza e formatação usando regex.

#### Importação de pacotes

In [ ]:
#!pip install unidecode

In [1]:
import glob
import os
import pandas as pd
import re
from unidecode import unidecode

import os
import zipfile
import io

import random

In [2]:
# VARIÁVEL GLOBAL PARA TESTAR VÁRIOS ENCODINGS
encoding = 'UTF-8'

#### Ler arquivos externos

In [3]:
def find_fpaths(dir_path, pattern):
    '''
    >> DESCRIÇÃO
    
    Usa o módulo glob para buscar todos os arquivos
    que correspondam ao padrão passado na variável 
    pattern'. Retorna uma lista de paths no formato 
    string. 
    
    >> PARÂMETROS
    
    dir_path -> uma string com o caminho para o
    diretório onde a busca pelos arquivos será
    realizada.
    
    pattern -> uma string com o padrão de texto
    que deve ser procurado no diretório.
    
    '''
    
    full_pattern = dir_path + pattern
    files = glob.glob(full_pattern)
    return files

In [20]:
def read_file(file_list, zip_file = False):
    '''
    >> DESCRIÇÃO
    
    Lê a lista de arquivos e configura o conteúdo
    em um dataframe com os seguintes campos:
    PRESIDENTE | CONTEUDO | ARQUIVO | ANO
    Funciona para os discursos em plenário
    
    >> PARÂMETROS
    
    file_list -> uma lista de filepaths em formato
    string. Ela é gerada anteriormente, na função
    find_fpaths.
    
    '''
    print('Reading Files...')
    content = []
    name_file_list = []
    if zip_file:
        for f in file_list:#[:5]:
            with zipfile.ZipFile(f) as zip_file:
                print(f, len(zip_file.filelist))
                ## serão selecionados apenas 5% do total de cada arquivo
                sample_size = round(len(zip_file.filelist)*.3)
                for name_file in random.sample(zip_file.filelist,sample_size):
                    name_file_list.append(unidecode(name_file.filename))
                    with io.TextIOWrapper(zip_file.open(name_file.filename), encoding="utf-8") as arq:
                        content.append(arq.read())

    else:
        # Lê o conteúdo dos arquivos de texto na lista
        content = [ open(file, encoding = encoding).read() for file in file_list ]
        name_file_list = [ unidecode(file) for file in file_list ]

    return content, name_file_list

def make_df_plen(file_list, content):
    print('Making  DF plen...')
    
    # Transforma eum dataframe
    df = pd.DataFrame({
        'FILE'             : [ file for file in file_list ],
        'ORIGINAL_CONTENT' : [ item for item in content ],
        'CLEAN_CONTENT'    : [ unidecode(item) for item in content ],
        'SESSION_TYPE'     : [ re.search("\-([A-Z\s]+)\-", file).group(1) for file in file_list ],
        'SESSION_DATE'     : [ re.search("\d{8}", file).group(0) for file in file_list ]
    })
    
    df["SESSION_DATE"] = pd.to_datetime(df.SESSION_DATE, format = "%d%m%Y")
    df["MONTH"] = df.SESSION_DATE.dt.month
    df["YEAR"] = df.SESSION_DATE.dt.year

    return df

#### Funções de formatação e busca usando regex

In [5]:
def find_speakers(string):
    
    '''
    Essa função detecta o padrão de texto
    que antecede a fala de um deputado e
    retorna um objeto match (via re.find_all).
    Ele é útil para detectar QUANTOS deputados
    falaram em determinada string textual.
    '''
    
    pattern = "((O?\s?SR\.?\s+?)|(A?\s?SRA\.?\s+?))(\s+DEPUTADO|\s+DEPUTADA|\s+PRESIDENTE|\s+PRESIDENTA)?"
    regexp = re.compile(pattern)
    matches = re.findall(regexp, string)
    
    return matches

In [6]:
def extract_full_quote(clean_string, original_string):
    
    '''
    Essa função extrai todas as falas de Jair Bolsonaro
    em uma determinada string. O pattern de regex encontra,
    primeiro, uma fala qualquer do Presidente. Então, pega
    tudo que está entre essa fala e a fala de outro parlamentar 
    ou o fim do arquivo. Isso é necessário porque há arquivos
    que misturam a fala de vários parlamentares, geralmente
    quando estão envolvidos em uma discussão.
    
    Para fazer essa operação, são passados textos sem caracteres
    especiais unicode. Depois de feita a captura, usamos os índices
    das matches no regex para extrair o mesmo pedaço de texto
    na string original.

    '''
    
    #if "O SR. PRESIDENTE (Jair Bolsonaro)" in clean_string:
     #   pattern = "O SR\. PRESIDENTE (\(Jair Bolsonaro\))?(.*?)((O?\s?SR\.?\s+?)|(A?\s?SRA\.?\s+?)|$)"
      #  group_no = 2

    #else:
    #pattern  = "O?\s?SR\.?\s?(DEPUTADO)?\s+JAIR\s+BOLSONARO\s?(\((Bloco\/)?\w{2,}\s?\-\s?\w{2}[^)]+\))?(.*?)((O?\s?SR\.?\s+?)|(A?\s?SRA\.?\s+?)|$)"
    pattern = "((O?\s?SR\.?\s+?)|(A?\s?SRA\.?\s+?))(\s+DEPUTADO|\s+DEPUTADA|\s+PRESIDENTE|\s+PRESIDENTA)?"
    group_no = 4
        
    regexp   = re.compile(pattern, re.MULTILINE)
    matches  = re.finditer(regexp, clean_string)
    
    full_clean_quote    = [ ]
    full_original_quote = [ ]
    
    for match in matches:
                        
        match_start = match.start(group_no)
        match_end   = match.end(group_no)
            
        clean_quote = match[group_no]
        clean_quote = clean_quote.replace("- ", "")
        
        original_quote = original_string[match_start:match_end]
        original_quote = original_quote.replace("- ", "")
        
        full_clean_quote.append(clean_quote)
        full_original_quote.append(original_quote)
        
    full_clean_quote    = ' [ INTERRUPÇÃO ] '.join(full_clean_quote)
    full_original_quote = ' [ INTERRUPÇÃO ] '.join(full_original_quote)

    # Remove espaços múltiplos internos usando a operação join de lista
    full_clean_quote    = ' '.join(full_clean_quote.split()) 
    full_original_quote = ' '.join(full_original_quote.split()) 
    
    return full_clean_quote, full_original_quote

#### Funções para aplicar operações de regex no dataframe

In [7]:
def apply_count_speakers(row):
    
    '''
    Aplica, linha a linha, a função
    find_speakers(string)
    '''
    
    matches = find_speakers(row.CLEAN_CONTENT)
    speaker_count = len(matches)
    
    return pd.Series({ "SPEAKER_COUNT":speaker_count 
        })

In [8]:
def apply_extract_full_quote(row):
    
    '''
    Aplica, linha a linha, a função
    extract_full_quote(string)
    '''
    
    full_clean_quote, full_original_quote = extract_full_quote(row.CLEAN_CONTENT, row.ORIGINAL_CONTENT)
    return pd.Series({
        "FULL_CLEAN_QUOTE"    : full_clean_quote,
        "FULL_ORIGINAL_QUOTE" : full_original_quote
    })

#### Função que encapsula anteriores e roda a operação

In [9]:
def run_script(zip_file = False):
    '''
    zip_file -> indica se os arquivos estão compactados no formato ZIP.
    
    Executa as operações anteriores em ambos os bancos de dados
    (plenário e comissões), filtra entradas sem match, concatena
    ambos os dataframes e salva para arquivo csv.
    '''
    list_info_file = []
    list_speak = []
    
    if zip_file:
        list_speak, list_info_file = read_file( find_fpaths("../data/txts/plenario/", "*.zip"), zip_file)
    else:
        list_speak, list_info_file = read_file(find_fpaths("../data/txts/plenario/", "*.txt"),zip_file)
        
    df_plen = make_df_plen(list_info_file,list_speak)
    
    #As sessões do tipo HOMENAGEM são apenas registro de protocolo. 
    #Não contém transcrição de discursos.
    df_plen = df_plen[ df_plen.SESSION_TYPE != "HOMENAGEM" ]
    
    # Aplica funções para extrair discursos
    df_plen["SPEAKER_COUNT"] = df_plen.apply(apply_count_speakers, axis=1)
    
    return df_plen


,FILE,ORIGINAL_CONTENT,CLEAN_CONTENT,SESSION_TYPE,SESSION_DATE,MONTH,YEAR,SPEAKER_COUNT
0,11548-29052001-17h00-ORDEM DO DIA-094-3-51-O.txt,O SR. EURÍPEDES MIRANDA (Bloco/PDT-RO. Pela or...,O SR. EURIPEDES MIRANDA (Bloco/PDT-RO. Pela or...,ORDEM DO DIA,2001-05-29,5,2001,1
1,15346-28032001-17h08-ORDEM DO DIA-033-3-51-O.txt,O SR. IÉDIO ROSA (Bloco/PSB-RJ. Pela ordem. Se...,O SR. IEDIO ROSA (Bloco/PSB-RJ. Pela ordem. Se...,ORDEM DO DIA,2001-03-28,3,2001,1
2,25-27122001-22h46-ORDEM DO DIA-004-5-51-C.txt,O SR. PAUDERNEY AVELINO (Bloco/PFL-AM. Sem rev...,O SR. PAUDERNEY AVELINO (Bloco/PFL-AM. Sem rev...,ORDEM DO DIA,2001-12-27,12,2001,1
3,12263-17052001-11h28-ORDEM DO DIA-083-3-51-O.txt,O SR. ALBERTO FRAGA (PMDB-DF. Pela ordem. Sem ...,O SR. ALBERTO FRAGA (PMDB-DF. Pela ordem. Sem ...,ORDEM DO DIA,2001-05-17,5,2001,1
5,8764-15082001-18h02-COMUNICACOES PARLAMENTARES...,O SR. IVAN VALENTE (PT-SP. Pela ordem. Pronunc...,O SR. IVAN VALENTE (PT-SP. Pela ordem. Pronunc...,COMUNICACOES PARLAMENTARES,2001-08-15,8,2001,1


In [21]:
## LENDO OS DISCURSOS E SALVANDO EM UM DATAFRAME
df = run_script(True)

Reading Files...
../data/txts/plenario\2001.zip 19355
../data/txts/plenario\2002.zip 10353
../data/txts/plenario\2003.zip 23829
../data/txts/plenario\2004.zip 19028
../data/txts/plenario\2005.zip 20185
../data/txts/plenario\2006.zip 15522
../data/txts/plenario\2007.zip 25116
../data/txts/plenario\2008.zip 20488
../data/txts/plenario\2009.zip 24499
../data/txts/plenario\2010.zip 14961
../data/txts/plenario\2011.zip 22727
../data/txts/plenario\2012.zip 20031
../data/txts/plenario\2013.zip 27752
../data/txts/plenario\2014.zip 17303
../data/txts/plenario\2015.zip 28169
../data/txts/plenario\2016.zip 23092
../data/txts/plenario\2017.zip 28134
../data/txts/plenario\2018.zip 16560
../data/txts/plenario\2019.zip 19506
Making  DF plen...


AttributeError: 'NoneType' object has no attribute 'group'

In [ ]:
print('Qtd de Registros por ANO:\n{}\n\nQtd Total de registros:{} '.format(df.YEAR.value_counts(),len(df)))

In [32]:
# Salva
directory = "../data/csvs/"
if not os.path.exists(directory):
    os.makedirs(directory)

name_file = directory + 'discursos_plen.csv'#str(df.YEAR.unique()[0]) + '_plen.csv'
df.to_csv(name_file, index = False)